In [8]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']

# Define the newspaper collection IDs
NEWSPAPER_COLLECTION_IDS = {
    'Dagens nyheter': 'https://libris.kb.se/m5z2w4lz3m2zxpk#it',
    'Svenska Dagbladet': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',
    'Aftonbladet': 'https://libris.kb.se/dwpgqn5q03ft91j#it',
    'Dagligt Allehanda': 'https://libris.kb.se/9tmqzv3m32xfzcz#it'
}

# Get the correct collection ID for the specified newspaper
collection_id = NEWSPAPER_COLLECTION_IDS.get(newspaper)
if not collection_id:
    raise ValueError(f"Invalid newspaper name: {newspaper}")

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Load checkpoint if it exists
checkpoint = load_checkpoint()
start_year = checkpoint['year'] if checkpoint else start_year
start_half = checkpoint['half'] if checkpoint else 0
start_index = checkpoint['index'] if checkpoint else 0

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print(f"Collection ID: {collection_id}")

# Load the venue list
df = pd.read_excel(venue_list)

Configuration Settings:
venue_list: Datasets/Venues_plus_Konsert.xlsx
start_year: 1908
years_to_crawl: 1
newspaper: Dagligt Allehanda
prompt_filepath: llm_prompt_for_deployment.txt
db_path: Datasets/26.07_Deployment_Version_1.0.db
llm_model: gpt-4o-mini
max_tokens: 1000
Stockholm_Concert_Database_Path: Datasets/All_Concerts_1908_filtered.xlsx
columns_to_compare: ['normalized_date', 'name', 'venue']
column_mapping: {'konsert_datum': 'date', 'konsert_namn': 'name', 'lokal_namn': 'venue', 'arrangör': 'organiser'}
Collection ID: https://libris.kb.se/9tmqzv3m32xfzcz#it


In [9]:
# Load checkpoint if it exists
checkpoint = load_checkpoint()
start_year = checkpoint['year'] if checkpoint else start_year
start_half = checkpoint['half'] if checkpoint else 0
start_index = checkpoint['index'] if checkpoint else 0

# Main loop
for year in range(start_year, start_year + years_to_crawl):
    for half in range(start_half, 2):
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        print(f"Processing data from {from_date} to {to_date}")

        for index in range(start_index, len(df)):
            row = df.iloc[index]
            query = row['Lokal']

            try:
                result = fetch_newspaper_data(
                    query=query,
                    from_date=from_date.strftime('%Y-%m-%d'),
                    to_date=to_date.strftime('%Y-%m-%d'),
                    newspaper=collection_id,
                    config=config,
                    db_path=db_path
                )

                if result.get('success'):
                    print(f"Processed query '{query}' successfully.")
                else:
                    print(f"Failed to process query '{query}': {result.get('message')}")

                # Save checkpoint after each query, successful or not
                save_checkpoint(year, half, index + 1)

            except Exception as e:
                print(f"Error processing query '{query}': {str(e)}")
                save_checkpoint(year, half, index)
                # Consider raising specific exceptions here instead of a broad Exception

        print(f"Waiting. Currently at {from_date} to {to_date}")
        time.sleep(3)  # in seconds

        # Reset start_index for the next half, but not start_half or start_year
        start_index = 0

    # Reset start_half for the next year, but not start_year
    start_half = 0

print("All queries processed for all years")

2024-07-25 14:21:06,346 - INFO - Starting fetch_newspaper_data for query: Konsert, dates: 1908-01-01 to 1908-06-30
2024-07-25 14:21:06,475 - INFO - Search results received. Hits: 0
2024-07-25 14:21:06,475 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:06,476 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:06,477 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:06,478 - INFO - Starting fetch_newspaper_data for query: La Croix salong, dates: 1908-01-01 to 1908-06-30


Processing data from 1908-01-01 00:00:00 to 1908-06-30 00:00:00
Processed query 'Konsert' successfully.
Checkpoint saved: Year 1908, Half 0, Index 1


2024-07-25 14:21:06,624 - INFO - Search results received. Hits: 0
2024-07-25 14:21:06,624 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:06,625 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:06,625 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:06,626 - INFO - Starting fetch_newspaper_data for query: Norra paviljongen i Trädgårdsföreningens lokal, dates: 1908-01-01 to 1908-06-30


Processed query 'La Croix salong' successfully.
Checkpoint saved: Year 1908, Half 0, Index 2


2024-07-25 14:21:06,832 - INFO - Search results received. Hits: 0
2024-07-25 14:21:06,832 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:06,834 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:06,834 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:06,836 - INFO - Starting fetch_newspaper_data for query: Wallmans lokal (Mäster Samuels gränd 11), dates: 1908-01-01 to 1908-06-30


...

2024-07-25 14:21:19,017 - INFO - Search results received. Hits: 0
2024-07-25 14:21:19,017 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:19,018 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:19,019 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:19,020 - INFO - Starting fetch_newspaper_data for query: Frimurarebarnhuset vid Christineberg, dates: 1908-01-01 to 1908-06-30


Processed query 'Piperska Muren' successfully.
Checkpoint saved: Year 1908, Half 0, Index 79


2024-07-25 14:21:19,279 - INFO - Search results received. Hits: 0
2024-07-25 14:21:19,280 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:19,281 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:19,281 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:19,282 - INFO - Starting fetch_newspaper_data for query: Dalarö kyrka, dates: 1908-01-01 to 1908-06-30
2024-07-25 14:21:19,412 - INFO - Search results received. Hits: 0
2024-07-25 14:21:19,413 - INFO - Extracted 0 URLs from search results
2024-07-25 14:21:19,413 - INFO - Table 'newspaper_data' created or already exists
2024-07-25 14:21:19,415 - INFO - Data processing completed. Total rows saved: 0
2024-07-25 14:21:19,417 - INFO - Starting fetch_newspaper_data for query: Tomteboda, strax bortom Carlberg, dates: 1908-01-01 to 1908-06-30


Processed query 'Frimurarebarnhuset vid Christineberg' successfully.
Checkpoint saved: Year 1908, Half 0, Index 80
Processed query 'Dalarö kyrka' successfully.
Checkpoint saved: Year 1908, Half 0, Index 81


KeyboardInterrupt: 

# Load SQL Database into Dataframe

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
display(df.head())


# Optional: If you want to see all rows, you can use:
# pd.set_option('display.max_rows', None)
# print(df)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)

# Experimental: Delete Duplicate Rows based on ComposedBlock Content

In [ ]:
import sqlite3
from sqlalchemy import create_engine

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
engine = create_engine(f'sqlite:///{db_path}', echo=False)

# Remove duplicates based on '[ComposedBlock Content]' column
df = df.drop_duplicates(subset='[ComposedBlock Content]')

# Update the existing table with the deduplicated DataFrame
df.to_sql('newspaper_data', engine, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"Deduplicated data committed to the 'newspaper_data' table. Rows: {len(df)}")
